# Demonstrate ``TuneAxis``

In this example, we demonstrate the `apstools.plans.TuneAxis()` plan.  The `TuneAxis()` support may be used to align (a.k.a. *tune*) a signal against an axis.

We'll use a software-only (not connected to hardware) motor as a positioner.  Here, we prepare a signal that is a computation based on the value of our positioner.  The computed signal is a model of a realistic diffraction peak ([pseudo-Voigt](https://en.wikipedia.org/wiki/Voigt_profile), a mixture of a Gaussian and a Lorentzian) one might encounter in a powder diffraction scan.  The model peak is a pseudo-voigt function to which some noise has been added.  Random numbers are used to modify the ideal pseudo-voigt function so as to simulate a realistic signal.

For this demo, we do not need the databroker since we do not plan to review any of this data after collection.  We'll display the data during the scan using the *BestEffortCallback()* code.

In [1]:
from ophyd import EpicsMotor
from apstools.synApps_ophyd import swaitRecord, swait_setup_random_number
from apstools.signals import SynPseudoVoigt
from apstools.plans import TuneAxis
from bluesky.callbacks import LiveTable
import numpy as np

from bluesky import RunEngine
RE = RunEngine({})

Figure out which workstation we are running.  The *mint-vm* host has a different IOC prefix.

In [2]:
import socket
if socket.gethostname().find("mint-vm") >= 0:
    prefix = "vm7:"
else:
    prefix = "xxx:"

Connect to our motor *before* we create the simulated detector signal.

In [3]:
m1 = EpicsMotor(prefix+"m1", name="m1")
m1.wait_for_connection()

Define a starting position, we'll use this later in the demo.

In [4]:
m1.move(-1.5)
starting_position = m1.position

## Setup the simulated detector signal.  

Randomize the values a bit so that we have something interesting to find with `TuneAxis()`.

In [5]:
spvoigt = SynPseudoVoigt(
    'spvoigt', m1, 'm1', 
    center=-1.5 + 0.4*np.random.uniform(), 
    eta=0.2 + 0.5*np.random.uniform(), 
    sigma=0.001 + 0.05*np.random.uniform(), 
    scale=1e5,
    bkg=0.01*np.random.uniform())

Reveal the actual values.  These are the answers we expect to discover.

In [6]:
print("spvoigt.scale: ", spvoigt.scale)
print("spvoigt.center: ", spvoigt.center)
print("spvoigt.sigma: ", spvoigt.sigma)
print("spvoigt.eta: ", spvoigt.eta)
print("spvoigt.bkg: ", spvoigt.bkg)

spvoigt.scale:  100000.0
spvoigt.center:  -1.308416833562333
spvoigt.sigma:  0.03901796340367001
spvoigt.eta:  0.5323629591618759
spvoigt.bkg:  0.0055871974459740905


We will add the actual values as metadata to these scans.

In [7]:
md = dict(
    activity = "TuneAxis development and testing",
    peak_model = "pseudo Voigt",
    peak_scale = spvoigt.scale,
    peak_center = spvoigt.center,
    peak_sigma = spvoigt.sigma,
    peak_eta = spvoigt.eta,
    peak_bkg = spvoigt.bkg
    )

## Set up the tuner

Create a *TuneAxis()* object.  The *tuner* needs to know the positioner, what range to scan to find the peak, *and* it needs the name of the signal to be scanned (since the signal list may have more than one signal).

In [8]:
tuner = TuneAxis([spvoigt], m1, signal_name=spvoigt.name)
tuner.width = 2.5
tuner.step_factor = tuner.num/2.5

Configure the *LiveTable* to also show the simulated detector signal.

In [9]:
live_table = LiveTable(["m1", "spvoigt"])
#spvoigt.read_attrs = ["value"]

# Multi-pass tune

Execute multiple passes to refine the centroid determination.
Each subsequent pass will reduce the width of the next scan by ``step_factor``.

In [10]:
RE(tuner.multi_pass_tune(), live_table, md=md)

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 16:50:45.2 |   -2.75000 |    597.691 |
|         2 | 16:50:45.7 |   -2.47000 |    618.719 |
|         3 | 16:50:46.2 |   -2.19000 |    662.798 |
|         4 | 16:50:46.7 |   -1.92000 |    774.525 |
|         5 | 16:50:47.2 |   -1.64000 |   1285.796 |
|         6 | 16:50:47.7 |   -1.36000 |  39448.673 |
|         7 | 16:50:48.2 |   -1.08000 |   2068.071 |
|         8 | 16:50:48.7 |   -0.81000 |    882.983 |
|         9 | 16:50:49.2 |   -0.53000 |    692.140 |
|        10 | 16:50:49.7 |   -0.25000 |    630.969 |
+-----------+------------+------------+------------+
generator TuneAxis.multi_pass_tune ['6e2ddcf1'] (scan num: 1)
x : m1
y : spvoigt
cen : -1.3585086753074413
com : -1.3708812399969512
fwhm : 0.2880312584860971
min : [  -2.75        597.69056825]
max : [ -1.36000000e+00   3.94486732e+04]
crossings : [-1.50252

('6e2ddcf1-906e-49de-90eb-dc5f44d535eb',
 '221fbd1f-562c-44d8-babe-74c555111edf',
 'cda248cc-58c0-4212-91ec-23f7034da210',
 '5dd62793-496a-4983-8f25-e08c3c13c11c')

Show the results from the multi-pass tuning.

In [11]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
for stat in tuner.stats:
    print("--", stat.cen, stat.fwhm)
print("m1=", m1.position, "", "det=", spvoigt.value)

final:  -1.30847406081
max (-1.3100000000000001, 100432.73883675525)
min (-1.3300000000000001, 81451.638145309073)
-- -1.35850867531 0.288031258486
-- -1.31155612165 0.110171766685
-- -1.30847406081 0.0769087300845
-- -1.30852729059 0.0280925714101
m1= -1.31  det= 81451.6381453


Compare the final position (just printed) with the expected value shown a couple steps back.

## Single-pass tune

Repeat but with only one pass.  Reset the motor to the starting position and increase the number of steps by a factor of three.

In [12]:
m1.move(starting_position)
tuner.num *= 3
RE(tuner.tune(), live_table, md=md)

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |    spvoigt |
+-----------+------------+------------+------------+
|         1 | 16:51:00.6 |   -2.75000 |    597.691 |
|         2 | 16:51:00.9 |   -2.66000 |    603.049 |
|         3 | 16:51:01.2 |   -2.58000 |    608.797 |
|         4 | 16:51:01.5 |   -2.49000 |    616.707 |
|         5 | 16:51:01.8 |   -2.41000 |    625.425 |
|         6 | 16:51:02.1 |   -2.32000 |    637.804 |
|         7 | 16:51:02.4 |   -2.23000 |    653.975 |
|         8 | 16:51:02.7 |   -2.15000 |    672.905 |
|         9 | 16:51:03.0 |   -2.06000 |    701.811 |
|        10 | 16:51:03.3 |   -1.97000 |    743.247 |
|        11 | 16:51:03.6 |   -1.89000 |    797.261 |
|        12 | 16:51:03.9 |   -1.80000 |    892.005 |
|        13 | 16:51:04.2 |   -1.72000 |   1032.892 |
|        14 | 16:51:04.5 |   -1.63000 |   1331.051 |
|        15 | 16:51:04.8 |   -1.54000 |   2028.211 |
|        16 | 16:51:05.1 |   -1.46000 |   3891

('e2fd6389-e617-4db5-a7b6-13b80a641e7d',)

Compare the single-pass scan with the previous multi-pass scan.  Each used the same number of points overall.  

The results are comparable but we already knew the position of the peak approximately.

In [13]:
print("final: ", tuner.center)
print("max", tuner.peaks.max)
print("min", tuner.peaks.min)
print("centroid", tuner.peaks.cen)
print("FWHM", tuner.peaks.fwhm)
print("m1=", m1.position, "", "det=", spvoigt.value)

final:  -1.29565829791
max (-1.28, 71213.880680860384)
min (-2.75, 597.6905682482369)
centroid -1.29565829791
FWHM 0.120189700489
m1= -1.3  det= 630.969061811
